In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import strategy_vgg_pretrained_model; reload(strategy_vgg_pretrained_model)
from keras.models import Sequential
import file_utils; reload(file_utils)
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.np_utils import to_categorical

In [95]:
% rm -rf ../data/kaggle_fisheries/output
% mkdir ../data/kaggle_fisheries/output
% cp -R  ../data/kaggle_fisheries/train ../data/kaggle_fisheries/output/train
%run createSample.py

## Strategy 1 : Train all Dense last layers of VGG Model

In [2]:
num = 64
path = "../data/kaggle_fisheries/output/sample/"
rootPath = "../data/kaggle_fisheries/output/"
batchGenerator_train = file_utils.fileBatchGenerator(rootPath+"train/", num)
batchGenerator_valid = file_utils.fileBatchGenerator(rootPath+"valid/", num)

Found 2395 images belonging to 8 classes.
Found 261 images belonging to 8 classes.


### The full VGG Model with its last layer replaced , training it first to form a base line

In [24]:
vggModel = strategy_vgg_pretrained_model.createVGGModelAllBlock()
strategy_vgg_pretrained_model.modifyAnyModelFor8Output(vggModel, False)
vggModel.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
vggModel.fit_generator(batchGenerator_train, samples_per_epoch=batchGenerator_train.nb_sample, nb_epoch=3,
                validation_data=batchGenerator_valid, nb_val_samples=batchGenerator_valid.nb_sample)

Epoch 1/3
2395/2395 [==============================] - 96s - loss: 2.4218 - acc: 0.4326 - val_loss: 1.0117 - val_acc: 0.6858

### Get The convolutional Features for faster training

In [11]:
vggModelConvolutionBlock = strategy_vgg_pretrained_model.getConvModelFromFullVGGModel(vggModel)
conv_features_valid = vggModelConvolutionBlock.predict_generator(batchGenerator_valid, batchGenerator_valid.nb_sample )
conv_features_train= vggModelConvolutionBlock.predict_generator(batchGenerator_train, batchGenerator_train.nb_sample )

In [12]:
file_utils.saveArray(path +"conv_features_valid", conv_features_valid)
file_utils.saveArray(path +"conv_features_train", conv_features_train)

### Setting up the Dense Layers for Training 

In [13]:
conv_features_valid = file_utils.loadArray(path +"conv_features_valid")
conv_features_train = file_utils.loadArray(path +"conv_features_train")
#strategy_vgg_pretrained_model.markLayersAsTrainable(vggModelDenseBlock.layers, True)
opt = Adam(lr=0.001)
one_hot_encoded_labels_validation = to_categorical(batchGenerator_valid.classes)
one_hot_encoded_labels_training = to_categorical(batchGenerator_train.classes)

### When all the dense layers are set to Trainable

In [14]:
vggModelDenseBlock = strategy_vgg_pretrained_model.getFCModelFromFullVGGModel(vggModel, 8)
vggModelDenseBlock.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vggModelDenseBlock.fit(conv_features_train, one_hot_encoded_labels_training, batch_size = 64, nb_epoch=10, validation_data=(conv_features_valid, one_hot_encoded_labels_validation) )

Train on 2395 samples, validate on 261 samples
Epoch 1/10
2395/2395 [==============================] - 4s - loss: 8.9985 - acc: 0.4388 - val_loss: 8.7075 - val_acc: 0.4598 128/2395 [>.............................] - ETA: 5s - loss: 10.9665 - acc: 0.2656

### When all but last the dense layers are set to NOT Trainable

In [15]:
vggModelDenseBlock = strategy_vgg_pretrained_model.getFCModelFromFullVGGModel(vggModel, 8)
strategy_vgg_pretrained_model.modifyAnyModelFor8Output(vggModelDenseBlock, False)
vggModelDenseBlock.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vggModelDenseBlock.fit(conv_features_train, one_hot_encoded_labels_training, batch_size = 64, nb_epoch=10, validation_data=(conv_features_valid, one_hot_encoded_labels_validation) )

Train on 2395 samples, validate on 261 samples
Epoch 1/10
2395/2395 [==============================] - 1s - loss: 9.0879 - acc: 0.4230 - val_loss: 8.7075 - val_acc: 0.4598 128/2395 [>.............................] - ETA: 1s - loss: 11.6998 - acc: 0.1797

### Using the VGG Model and manually setting its last few layers to trainable

In [27]:
indexOfLastConvolutedLayer = strategy_vgg_pretrained_model.setFCLayersToTrainable(vggModel)
vggModel.compile(optimizer=Adam(lr=0.00001),
                loss='categorical_crossentropy', metrics=['accuracy'])
print(vggModel.layers[indexOfLastConvolutedLayer], vggModel.layers[indexOfLastConvolutedLayer].trainable)
print(vggModel.layers[indexOfLastConvolutedLayer+1], vggModel.layers[indexOfLastConvolutedLayer+1].trainable)

(<keras.layers.convolutional.Convolution2D object at 0x7f3dbd722f10>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7f3dc120ce50>, True)


In [ ]:
vggModel.fit_generator(batchGenerator_train, samples_per_epoch=batchGenerator_train.nb_sample, nb_epoch=3,
                validation_data=batchGenerator_valid, nb_val_samples=batchGenerator_valid.nb_sample)

Epoch 1/3
  64/2395 [..............................] - ETA: 141s - loss: 11.5849 - acc: 0.2812